In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
df = pd.read_csv('data/bilateral_trade_clean.csv')
# let's print all the columns
pd.set_option('display.max_columns', None)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/bilateral_trade_clean.csv'

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
#lets drop a few columns
# cd = df.drop(columns=["partner2ISO","classificationCode","isReported","product_desc","qty","qtyUnitCode","qtyUnitAbbr","grossWgt","isQtyEstimated","isAltQtyEstimated","isNetWgtEstimated","isGrossWgtEstimated","aggrLevel","isLeaf"])
cd = df[['year', 'flowCode', 'reporterISO', 'partnerISO', 'netWgt_kg', 'fobvalue', 'unit_value_usd_per_kg']]

In [ ]:
# cd.head()
cd.shape

In [ ]:
#lets calculate unit price for fob and replace this column by unit_value_usd_per_kg
cd['unit_value_usd_per_kg'] = cd['fobvalue']/cd['netWgt_kg']
cd.head()

Different Flow codes

In [ ]:
cd['flowCode'].unique()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18,5))
sns.boxplot(y=cd['unit_value_usd_per_kg'], ax=axes[0])
axes[0].set_title("Unit Price (USD/kg)")
sns.boxplot(y=cd['netWgt_kg'], ax=axes[1])
axes[1].set_title("Net Weight (kg)")
sns.boxplot(y=cd['fobvalue'], ax=axes[2])
axes[2].set_title("FOB Value (USD)")
plt.suptitle("Before Outlier Removal", fontsize=14)
plt.show()

Take a log

In [ ]:
#drop values where netWgt_kg is 0.0
cd = cd[cd['netWgt_kg'] >= 1.0]
cd.sort_values(by='netWgt_kg').head()

In [ ]:
# cd.sort_values(by='netWgt_kg').head(100)

In [ ]:
# cd_filtered = cd[cd['netWgt_kg'] >= 1].copy()
plt.figure(figsize=(8,6))
sns.boxplot(y=cd['netWgt_kg'])
plt.title("Net Weight (kg) Distribution (filtered)")
plt.ylabel("Net Weight (kg)")
plt.yscale('log') # Use a log scale for better visualization due to wide range
plt.show()

In [ ]:
#countries that pay more for per unit price
cd = cd[cd['unit_value_usd_per_kg'] <= 100.0]

# Find 99th percentile cutoff
upper = cd['unit_value_usd_per_kg'].quantile(0.99)

# Keep all rows below that cutoff
cd = cd[cd['unit_value_usd_per_kg'] <= upper].reset_index(drop=True)

cd.sort_values(by='unit_value_usd_per_kg', ascending=False).head()



In [ ]:
#box plot of unit_value_usd_per_kg
plt.figure(figsize=(8,6))
sns.boxplot(y=cd['unit_value_usd_per_kg'])
plt.title("Unit Price (USD/kg) Distribution")
plt.ylabel("Unit Price (USD/kg)")
plt.yscale('log') # Use a log scale for better visualization due to wide range
plt.show()

In [ ]:
cd.describe()

In [ ]:
# cd.sort_values(by='unit_value_usd_per_kg', ascending=False).head(100)

In [ ]:
# can you save this data frame to csv or maybe interactive google sheet?
# cd.to_csv('cd.csv', index=False)

In [ ]:
# fig, axes = plt.subplots(1, 3, figsize=(18,5))
# sns.boxplot(y=cd['unit_value_usd_per_kg'], ax=axes[0])
# axes[0].set_title("Unit Price (USD/kg)")
# sns.boxplot(y=cd['netWgt_kg'], ax=axes[1])
# axes[1].set_title("Net Weight (kg)")
# sns.boxplot(y=cd['fobvalue'], ax=axes[2])
# axes[2].set_title("FOB Value (USD)")
# plt.suptitle("After Outlier Removal", fontsize=14)
# plt.show()

fig, axes = plt.subplots(1, 3, figsize=(18,5))

sns.boxplot(y=np.log1p(cd['unit_value_usd_per_kg']), ax=axes[0])
axes[0].set_title("Unit Price (log USD/kg)")

sns.boxplot(y=np.log1p(cd['netWgt_kg']), ax=axes[1])
axes[1].set_title("Net Weight (log kg)")

sns.boxplot(y=np.log1p(cd['fobvalue']), ax=axes[2])
axes[2].set_title("FOB Value (log USD)")

plt.suptitle("After Outlier Removal (Log Transformed)", fontsize=14)
plt.show()

Convert all values to string

In [ ]:
cd['netWgt_kg'] = pd.to_numeric(cd['netWgt_kg'], errors='coerce')
cd['unit_value_usd_per_kg'] = pd.to_numeric(cd['unit_value_usd_per_kg'], errors='coerce')
cd['fobvalue'] = pd.to_numeric(cd['fobvalue'], errors='coerce')
cd = cd.dropna(subset=['year', 'reporterISO', 'partnerISO'])
cd['year'] = cd['year'].astype(int)

In [ ]:
# grouped = cd.groupby(['partnerISO', 'year']).agg({
#     'netWgt_kg': 'sum',
#     'unit_value_usd_per_kg': 'mean'
# }).reset_index()

In [ ]:
# Filter out partner = 'World' before grouping
cd = cd[cd['partnerISO'] != 'World']

# # Then re-group
grouped = cd.groupby(['partnerISO', 'year']).agg({
    'netWgt_kg': 'sum',
    'unit_value_usd_per_kg': 'mean'
}).reset_index()

plotly library

do it size as net weight

In [ ]:
# Assuming your dataframe is named 'grouped'
fig = px.scatter(
    grouped,
    x="netWgt_kg",
    y="unit_value_usd_per_kg",
    size="netWgt_kg",  # bubble size
    color="partnerISO",  # color by partner country
    hover_name="partnerISO",
    hover_data={"netWgt_kg": ":,.0f", "unit_value_usd_per_kg": ":.3f", "year": True},
    size_max=60,  # max bubble size
    opacity=0.4,
    title="Trade Volume vs Unit Price Across Countries (Bubble Size = Net Weight)",
    labels={
        "netWgt_kg": "Net Weight (kg)",
        "unit_value_usd_per_kg": "Unit Price (USD/kg)",
        "partnerISO": "Importing Country"
    },
    log_x=True,
    log_y=True,
)

fig.update_layout(
    title_font=dict(size=18),
    xaxis=dict(title="Net Weight (kg, log scale)"),
    yaxis=dict(title="Unit Price (USD/kg, log scale)"),
    legend_title_text="Partner Country",
    template="plotly_white",
    height=700,
)

fig.show()

In [ ]:
import plotly.express as px

# Remove aggregated 'World' entries
# cd = cd[cd['partnerISO'] != 'World']

# Re-group by partner and year
grouped = cd.groupby(['partnerISO', 'year']).agg({
    'netWgt_kg': 'sum',
    'unit_value_usd_per_kg': 'mean'
}).reset_index()

# --- Interactive bubble chart with year animation ---
fig = px.scatter(
    grouped,
    x="netWgt_kg",
    y="unit_value_usd_per_kg",
    size="netWgt_kg",  # bubble size = total trade volume
    color="partnerISO",  # color by importer
    hover_name="partnerISO",
    hover_data={
        "year": True,
        "netWgt_kg": ":,.0f",
        "unit_value_usd_per_kg": ":.3f"
    },
    animation_frame="year",   # adds year animation
    size_max=60,
    opacity=0.7,
    title="Trade Volume vs Unit Price Over Time (Bubble Size = Net Weight, Excluding 'World')",
    labels={
        "netWgt_kg": "Net Weight (kg, log scale)",
        "unit_value_usd_per_kg": "Unit Price (USD/kg)",
        "partnerISO": "Importing Country"
    },
    log_x=True,
    log_y=True,
)

fig.update_layout(
    template="plotly_white",
    height=700,
    legend_title_text="Partner Country",
    title_font=dict(size=18),
)

fig.show()

In [ ]:
cd.head()

In [ ]:
cd.shape

Machine Learning

In [ ]:
mdf = cd.copy()
# mdf.head()

In [ ]:
# Sorting it with year and changing the year to data time datatype
mdf = mdf.sort_values(by='year', ascending=False)
mdf['year'] = pd.to_datetime(mdf['year'], format='%Y').dt.year.astype(int)
mdf.head()

In [ ]:
# creating dummies of flowcode as the feature is cateogirical
mdf = pd.get_dummies(mdf, columns=['flowCode'])
mdf.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# --- Step 1: Aggregate at the global level (sum or mean by year)
world = (
    mdf.groupby('year')
       .agg(
           total_weight=('netWgt_kg', 'sum'),
           total_fob=('fobvalue', 'sum'),
           avg_unit_value=('unit_value_usd_per_kg', 'mean')
       )
       .reset_index()
)

sns.set(style="whitegrid")

# 1️⃣ Total Global Export Volume
plt.figure(figsize=(10,6))
sns.lineplot(data=world, x='year', y='total_weight')
plt.title('Global Total Export Volume Over Time')
plt.ylabel('Total Weight (kg)')
plt.yscale('log')  # log scale makes large changes easier to read
plt.tight_layout()
plt.show()

# 2️⃣ Total Global FOB Value
plt.figure(figsize=(10,6))
sns.lineplot(data=world, x='year', y='total_fob')
plt.title('Global Total FOB Value Over Time')
plt.ylabel('FOB Value (USD)')
plt.yscale('log')
plt.tight_layout()
plt.show()

# 3️⃣ Global Average Unit Value (USD/kg)
plt.figure(figsize=(10,6))
sns.lineplot(data=world, x='year', y='avg_unit_value')
plt.title('Global Average Unit Value (USD/kg) Over Time')
plt.ylabel('USD per kg')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import plotly.express as px

# --- Step 1️⃣ Group by year and reporter country ---
agg = (
    mdf.groupby(['year', 'reporterISO'])
       .agg(
           total_weight=('netWgt_kg', 'sum'),
           total_fob=('fobvalue', 'sum'),
           avg_unit_value=('unit_value_usd_per_kg', 'mean')
       )
       .reset_index()
)

# --- Step 2️⃣ Find top 15 exporters by total trade volume (sum over all years) ---
top15 = (
    agg.groupby('reporterISO')['total_weight']
       .sum()
       .nlargest(15)
       .index
)

# --- Step 3️⃣ Filter the aggregated data to only include those top 15 ---
agg_top15 = agg[agg['reporterISO'].isin(top15)]

# --- Step 4️⃣ Create the interactive line chart ---
fig = px.line(
    agg_top15,
    x='year',
    y='total_weight',
    color='reporterISO',
    hover_name='reporterISO',
    markers=True,
    title='Top 15 Exporting Countries by Total Volume (kg)',
    labels={
        'year': 'Year',
        'total_weight': 'Total Export Volume (kg)',
        'reporterISO': 'Exporter Country'
    }
)

# --- Step 5️⃣ Optional styling (log scale helps with huge differences) ---
fig.update_yaxes(title_text="Total Export Volume (kg, log scale)", type='log')
fig.update_layout(
    template='plotly_white',
    legend_title_text='Exporter Country',
    height=600,
    title_font=dict(size=20)
)

# --- Step 6️⃣ Show the chart ---
fig.show()

In [ ]:
mdf.describe()

In [ ]:
# Train: up to 2020, Test: after 2020
train = mdf[mdf['year'] <= 2020]
test  = mdf[mdf['year'] > 2020]

# Define features and target
X_train = train[['year', 'netWgt_kg', 'fobvalue']]
y_train = train['unit_value_usd_per_kg']

X_test  = test[['year', 'netWgt_kg', 'fobvalue']]
y_test  = test['unit_value_usd_per_kg']

print(f"Train years: {train['year'].min()}–{train['year'].max()}, size={len(train)}")
print(f"Test  years: {test['year'].min()}–{test['year'].max()}, size={len(test)}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# --- Step 1: Aggregate global data by year ---
wdf = (
    mdf.groupby('year')
       .agg(
           total_weight=('netWgt_kg', 'sum'),
           total_fob=('fobvalue', 'sum'),
           avg_unit_value=('unit_value_usd_per_kg', 'mean')  # target variable
       )
       .reset_index()
)

# Sort by year
wdf = wdf.sort_values('year')

# --- Step 2: Create lag features (using past values only) ---
for i in [1, 2, 3]:
    wdf[f'total_weight_lag{i}'] = wdf['total_weight'].shift(i)
    wdf[f'total_fob_lag{i}'] = wdf['total_fob'].shift(i)
    wdf[f'price_lag{i}'] = wdf['avg_unit_value'].shift(i)

# Drop first 3 years (because they have missing lags)
wdf = wdf.dropna().reset_index(drop=True)

# --- Step 3: Define features and target ---
X = wdf[['total_weight_lag1', 'total_weight_lag2', 'total_weight_lag3',
         'total_fob_lag1', 'total_fob_lag2', 'total_fob_lag3',
         'price_lag1', 'price_lag2', 'price_lag3']]
y = wdf['avg_unit_value']

# --- Step 4: Time-based split ---
train = wdf[wdf['year'] <= 2020]
test  = wdf[wdf['year'] > 2020]

X_train = train[X.columns]
y_train = train['avg_unit_value']
X_test  = test[X.columns]
y_test  = test['avg_unit_value']

print(f"Train: {train['year'].min()}–{train['year'].max()}  |  Test: {test['year'].min()}–{test['year'].max()}")

# --- Step 5: Train model ---
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# --- Step 6: Evaluate ---
mae  = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nMAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")

# --- Step 7: Plot actual vs predicted ---
plt.figure(figsize=(10,6))
plt.plot(train['year'], y_train, label='Train (Actual)', marker='o')
plt.plot(test['year'], y_test, label='Test (Actual)', marker='o')
plt.plot(test['year'], y_pred, label='Predicted (2021–2024)', linestyle='--', marker='x')
plt.title('Global Average Export Price (USD/kg) — Lag-based Forecast (No Leakage)')
plt.xlabel('Year')
plt.ylabel('USD per kg')
plt.legend()
plt.tight_layout()
plt.show()